# Build Master Parcel Analysis Table (MPAT)
Author: Alemarie Ceria <br>
Last Updated: 02/24/26

## Setup

### Imports

In [3]:
from pathlib import Path
from datetime import datetime
from zoneinfo import ZoneInfo
import geopandas as gpd
import pandas as pd
import numpy as np

import arcpy
arcpy.env.overwriteOutput = True
print(arcpy.GetInstallInfo()["Version"])

# Load helper functions
%run ../src/build_mpat.py

3.6.1


### Configurations

In [4]:
# Current MPAT version
VERSION = "02"

# Today
TODAY = datetime.now(ZoneInfo("Pacific/Honolulu")).strftime("%Y%m%d")

# Pilot island(s) (set to None for all MHI)
PILOT_ISLANDS = ["Maui"]

# Target CRS
TARGET_CRS = 32604

# Area unit conversions
AREA_CONVERSIONS: dict[tuple[str, str], float] = {
    ("sqm", "sqm"):   1.0,
    ("sqm", "sqft"):  3.28084 ** 2,     # 10.7639...
    ("sqft", "sqft"): 1.0,
    ("sqft", "sqm"):  0.3048 ** 2,      # 0.09290304
}

# For distance calculations
FT_TO_M = 0.3048

# For raster value conversions
UNIT_CONVERSIONS: dict[tuple[str, str], float] = {
    ("m", "m"):   1.0,
    ("m", "ft"):  3.28084,
    ("ft", "ft"): 1.0,
    ("in", "in"): 1.0,
}

### Paths

In [5]:
# Project root (assumes notebook is in `project/notebooks/`)
project_root = Path.cwd().parent

# Directories
source_dir = project_root / "data" / "01_inputs" / "source"
prepared_dir = project_root / "data" / "01_inputs" / "prepared"
interim_dir = project_root / "data" / "02_interim"
output_dir  = project_root / "data" / "03_processed"

# Tempspace for intermediate Arc outputs
tempspace = interim_dir / "tempspace"
tempspace.mkdir(parents=True, exist_ok=True)

# Scratch geopackage for 64-bit integer support
scratch_gpkg = tempspace / "scratch.gpkg"

### Data Sources

In [6]:
# Prepared inputs
inputs = {
    # Vectors (GPKG files)
    "cesspools": str(prepared_dir / "cesspools_inventory_hi_hcpt_32604.gpkg"),
    "parcels": str(prepared_dir / "parcels_hi_higp_32604.gpkg"),
    "coastline": str(prepared_dir / "coastline_hi_op_32604.gpkg"),
    "sma": str(prepared_dir / "sma_hi_op_32604.gpkg"),
    "streams": str(prepared_dir / "streams_hi_hcpt_32604.gpkg"),
    "wells_dom": str(prepared_dir / "wells_dom_hi_hcpt_32604.gpkg"),
    "wells_mun": str(prepared_dir / "wells_mun_hi_hcpt_32604.gpkg"),
    "building_fps": str(prepared_dir / "building_footprints_maui_data_source_32604.gpkg"),
    "soils": str(prepared_dir / "soils_hi_hcpt_32604.gpkg"),
    "flood_zones": str(prepared_dir / "flood_zones_hi_op_32604.gpkg"),
    # Rasters (GeoTIFFs inside named subfolders)
    "rainfall": str(prepared_dir / "annual_rainfall_hi_hcpt_32604" / "annual_rainfall_hi_hcpt_32604.tif"),
    "watertable": str(prepared_dir / "watertable_hi_hcpt_mosaic_32604" / "watertable_hi_hcpt_mosaic_32604.tif"),
    "dem": str(prepared_dir / "dem_hi_pacioos_mosaic_32604" / "dem_hi_pacioos_mosaic_32604.tif"),
    "slope": str(prepared_dir / "slope_hi_hcpt_mosaic_32604" / "slope_hi_hcpt_mosaic_32604.tif"),
}

# Outputs
outputs = {
    "mpat_gpkg": output_dir / f"mpat_v{VERSION}_32604.gpkg",
    "mpat_csv":  output_dir / f"mpat_v{VERSION}.csv",
}

## Data Processing

### Cesspool Attributes

In [7]:
cesspools_df = (
    # Load layer
    gpd.read_file(inputs["cesspools"], layer="cesspools")
    # Standardize column names to lowercase
    .rename(columns=lambda col: col.lower())
    # Subset to relevant columns
    .loc[:, ["tmk", "island", "osds_qty", "bedroom", "class_iv"]]
    # Rename columns
    .rename(columns={"bedroom": "bedroom_qty"})
    .assign(
        # Convert tmk column to string, trim whitespace, drop trailing ".0"
        tmk=lambda d: (
            d["tmk"]
            .astype("string")
            .str.strip()
            .str.replace(r"\.0$", "", regex=True)
        ),
        # Replace -9999 values with NA, keep as nullable integer
        bedroom_qty=lambda d: (
            d["bedroom_qty"]
            .replace(-9999, pd.NA)
            .astype("Int64")
        ),
    )
    # Filter to only records with class IV systems and at least 1 OSDS
    .query("class_iv != 0 and osds_qty > 0")
    # Drop duplicates on TMK (keeps first occurrence)
    .drop_duplicates(subset=["tmk"], keep="first")
    # Filter to pilot islands (if specified)
    .query("island in @PILOT_ISLANDS" if PILOT_ISLANDS else "True")
    .reset_index(drop=True)
    .drop(columns=["class_iv"])
)
print(len(cesspools_df))
cesspools_df.head()

7517


,tmk,island,osds_qty,bedroom_qty
0,221019029,Maui,1,3
1,221019033,Maui,2,4
2,221019034,Maui,2,5
3,221019035,Maui,2,7
4,221019036,Maui,1,3


### Parcels

In [8]:
parcels_gdf = (
    # Load layer
    gpd.read_file(inputs["parcels"], layer="parcels")
    # Standardize column names to lowercase
    .rename(columns=lambda col: col.lower())
    # Subset to relevant columns
    .loc[:, ["tmk_txt", "geometry"]]
    # Rename columns
    .rename(columns={"tmk_txt": "tmk"})
    # Filter to parcels with cesspools
    .query("tmk in @cesspools_df.tmk")
    # Dissolve: 1 row per TMK (unions split polygons)
    .dissolve(by="tmk", as_index=False)
    # Fix invalid geometries before union/dissolve
    .assign(geometry=lambda d: d.geometry.make_valid())
    # Calculate parcel areas in sqm and sqft
    .assign(
        parcel_area_sqm=lambda d: d.geometry.area,
        parcel_area_sqft=lambda d: d.geometry.area * AREA_CONVERSIONS[("sqm", "sqft")],
    )
    # Relocate geometry column to the end
    .pipe(lambda d: d[[c for c in d.columns if c != "geometry"] + ["geometry"]])
)
print(len(parcels_gdf))
parcels_gdf.head()

7505


,tmk,parcel_area_sqm,parcel_area_sqft,geometry
0,211003003,6589.121816,70924.721493,"POLYGON ((797198.639 2309554.974, 797196.84 23..."
1,211003006,3005.091424,32346.536954,"POLYGON ((797225.075 2309335.433, 797159.619 2..."
2,211003012,3609.705090,38854.544707,"POLYGON ((797153.617 2309339.286, 797107.905 2..."
3,211003030,1122.902488,12086.822566,"POLYGON ((796828.023 2309063.058, 796822.193 2..."
4,211003031,12021.238740,129395.545175,"POLYGON ((796820.317 2309089.443, 796830.047 2..."


### Building Footprints

Footprint-level:

In [9]:
building_fps_gdf = (
    # Load layer
    gpd.read_file(inputs["building_fps"], layer="building_fps")
    # Standardize column names to lowercase
    .rename(columns=lambda col: col.lower())
    # Subset to relevant columns
    .loc[:, ["tmk", "geometry"]]
    # Filter to tmks in parcels_gdf
    .query("tmk in @parcels_gdf.tmk")
    # Force to 2d geometries
    .assign(geometry=lambda d: d.geometry.force_2d())
    # Calculate building footprint area
    .assign(
        # Calculate area in sqft
        building_fp_area_sqft=lambda d: d.geometry.area * AREA_CONVERSIONS[("sqm", "sqft")]
    )
    # Relocate geometry column to the end
    .pipe(lambda d: d[[c for c in d.columns if c != "geometry"] + ["geometry"]])
)
print(len(building_fps_gdf))
building_fps_gdf.head()

c:\Users\apc\AppData\Local\ESRI\conda\envs\matrix_env\Lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiPolygon' is converted to 'MultiPolygon Z'
  return ogr_read(
c:\Users\apc\AppData\Local\ESRI\conda\envs\matrix_env\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Non-conformant content for record 1 in column LASTUPDATE, 2024-11-27T00:00:00.0Z, successfully parsed
  return ogr_read(


14540


,tmk,building_fp_area_sqft,geometry
35,244008023,1728.356968,"MULTIPOLYGON (((740226.531 2315158.217, 740216..."
58,247010078,2477.163231,"MULTIPOLYGON (((746446.96 2306653.92, 746446.4..."
62,247010082,2817.545549,"MULTIPOLYGON (((746370.816 2306761.394, 746367..."
64,247010082,343.683554,"MULTIPOLYGON (((746372.56 2306785.54, 746372.4..."
65,247010082,5772.306440,"MULTIPOLYGON (((746341.541 2306791.415, 746339..."


Export building footprints per parcel for mapping:

In [10]:
building_fps_per_parcel_gdf = building_fps_gdf.query("tmk in @parcels_gdf.tmk")
print(len(building_fps_per_parcel_gdf))

name = f"{TODAY}_building_fps_per_parcel"

building_fps_per_parcel_gdf.to_file(
    interim_dir / f"{name}.gpkg",
    layer=name,
    driver="GPKG"
)

14540


Parcel-level:

In [11]:
building_fp_parcel_attrs_df = (
    building_fps_gdf
    # Aggregate to parcel level
    .groupby("tmk", as_index=False)
    # Calculate building footprint quantity and total area for each parcel
    .agg(
        # Count number of building footprints per parcel
        building_fp_qty=("geometry", "size"),
        # Sum of building footprint area in sqft per parcel
        building_fp_total_area_sqft=("building_fp_area_sqft", "sum")
    )
)
print(len(building_fp_parcel_attrs_df))
building_fp_parcel_attrs_df.head()

7199


,tmk,building_fp_qty,building_fp_total_area_sqft
0,211003003,2,3445.443843
1,211003006,2,2271.544397
2,211003012,2,1777.038765
3,211003030,1,661.600642
4,211003041,1,1617.058704


### Parcel Analysis Points

Per parcel:
- If multiple buildings > choose centroid of the largest footprint
- If one building > choose centroid of that footprint
- If no buildings > choose parcel centroid

In [12]:
# Get building footprint parcel summary attributes for analysis points
analysis_points_attrs_df = (
    building_fps_gdf
    # Aggregate to parcel level
    .groupby("tmk", as_index=False)
    # Calculate largest/smallest footprint area on each parcel
    .agg(
        # Largest building footprint area in sqft per parcel
        building_fp_largest_area_sqft=("building_fp_area_sqft", "max"),
        # Smallest building footprint area in sqft per parcel
        building_fp_smallest_area_sqft=("building_fp_area_sqft", "min"),
    )
    # Merge building footprint quantity per parcel column
    .merge(building_fp_parcel_attrs_df[["tmk", "building_fp_qty"]], on="tmk", how="left")
    .assign(
        # Create flag for parcels with more than 1 building footprint
        building_fp_multi_flag=lambda d: d["building_fp_qty"] > 1
    )
    # Convert boolean flags to integers (1 for True, 0 for False)
    .assign(
        building_fp_qty=lambda d: d["building_fp_qty"].astype("int64"),
        building_fp_multi_flag=lambda d: d["building_fp_multi_flag"].astype("int64"),
    )
)

# Largest building footprint centroid per parcel
largest_fp_centroids_gdf = (
    building_fps_gdf
    .sort_values(["tmk", "building_fp_area_sqft"], ascending=[True, False])
    .drop_duplicates(subset=["tmk"], keep="first")
    .assign(largest_fp_centroid=lambda d: d.geometry.centroid)
    .loc[:, ["tmk", "largest_fp_centroid"]]
)

# Parcel centroids (used if no building footprint exists on parcel)
parcel_centroids_gdf = (
    parcels_gdf.loc[:, ["tmk", "geometry"]]
    .assign(parcel_centroid=lambda d: d.geometry.centroid)
    .loc[:, ["tmk", "parcel_centroid"]]
)

# Create 1 point per parcel for analysis, using building footprint centroid if it exists, otherwise parcel centroid
analysis_points_gdf = (
    parcel_centroids_gdf
    .merge(largest_fp_centroids_gdf, on="tmk", how="left", validate="one_to_one")
    .assign(
        # If a footprint exists (1+), use centroid of largest footprint; else parcel centroid
        geometry=lambda d: d["largest_fp_centroid"].where(
            d["largest_fp_centroid"].notna(),
            d["parcel_centroid"]
        ),
        # Create metdata column to keep track of the logic
        analysis_point_source=lambda d: d["largest_fp_centroid"].notna().map(
            {True: "building_fp_largest_centroid", False: "parcel_centroid"}
        ),
    )
    # Drop helper geoms
    .drop(columns=["largest_fp_centroid", "parcel_centroid"])
    # Relocate geometry column to the end
    .pipe(lambda d: d[[c for c in d.columns if c != "geometry"] + ["geometry"]])
)
print(len(analysis_points_gdf))
analysis_points_gdf.head()

7505


,tmk,analysis_point_source,geometry
0,211003003,building_fp_largest_centroid,POINT (797177.171 2309494.086)
1,211003006,building_fp_largest_centroid,POINT (797183.645 2309328.508)
2,211003012,building_fp_largest_centroid,POINT (797122.531 2309322.53)
3,211003030,building_fp_largest_centroid,POINT (796821.542 2309061.456)
4,211003031,parcel_centroid,POINT (796803.834 2309178.192)


Export analysis points:

In [15]:
analysis_layer = f"{TODAY}_analysis_points"

# Write the layer
analysis_points_gdf.to_file(
    interim_dir / f"{analysis_layer}.gpkg",
    layer=analysis_layer,
    driver="GPKG"
)

# ArcPy cursor path to the layer inside the GPKG
analysis_fc = str(interim_dir / f"{analysis_layer}.gpkg") + "\\" + analysis_layer

# Does ArcPy see it?
print(arcpy.management.GetCount(analysis_fc))

7505


### Special Management Area (SMA)

In [17]:
# Load and clean SMA layer
sma_gdf = (
    # Load layer
    gpd.read_file(inputs["sma"], layer="sma")
    # Subset to geometry column only
    .loc[:, ["geometry"]]
    # Fix invalid geometries before spatial operations
    .assign(geometry=lambda d: d.geometry.make_valid())
)

# Create a single geometry representing the union of all SMA areas for distance calculations
sma_union = sma_gdf.geometry.union_all()

# Distance calculations from analysis points to nearest SMA area
dist_to_sma_df = (
    analysis_points_gdf.loc[:, ["tmk", "geometry"]]
    .assign(
        # Calculate distance from analysis point to nearest SMA area in meters
        dist_to_sma_m=lambda d: d.geometry.distance(sma_union),
        # Convert distance to feet
        dist_to_sma_ft=lambda d: d["dist_to_sma_m"] / FT_TO_M,
    )
    # Drop geometry column
    .drop(columns=["geometry"])
)
print(len(dist_to_sma_df))
print(f"Number of points within SMA: {len(dist_to_sma_df.query("dist_to_sma_ft == 0.0"))}")
dist_to_sma_df.head()

7505
Number of points within SMA: 931


,tmk,dist_to_sma_m,dist_to_sma_ft
0,211003003,0.0,0.0
1,211003006,0.0,0.0
2,211003012,0.0,0.0
3,211003030,0.0,0.0
4,211003031,0.0,0.0


Values of 0.0 mean the analysis point falls within an SMA. For points outside of an SMA, distances are calculated.

### Flood Zones

[Flood Zones Metadata](https://files.hawaii.gov/dbedt/op/gis/data/s_fld_haz_ar_state.pdf)

In [18]:
flood_zones_gdf = (
    # Load layer
    gpd.read_file(inputs["flood_zones"], layer="flood_zones")
    # Subset to relevant columns
    .loc[:, ["sfha_tf", "geometry"]]
    # Fix invalid geometries before spatial operations
    .assign(geometry=lambda d: d.geometry.make_valid())
    # Keep only Special Flood Hazard Area (SFHA) zones (zones with "sfha_tf" == True)
    .query("sfha_tf == 'T'")
    .drop(columns=["sfha_tf"])
)
print(len(flood_zones_gdf))

# Intersect analysis points with flood zone polygons
ap_flood_zone_join = gpd.sjoin(
    analysis_points_gdf.loc[:, ["tmk", "geometry"]],
    flood_zones_gdf.loc[:, ["geometry"]],
    how="left",
    predicate="intersects",
)

# Create in flood zone flag
flood_flag_df = (
    ap_flood_zone_join.assign(in_flood_zone=ap_flood_zone_join["index_right"].notna())
          .groupby("tmk", as_index=False)["in_flood_zone"].any()
          .assign(in_flood_zone=lambda d: d["in_flood_zone"].astype("int64"))
)

print(len(flood_flag_df))
flood_flag_df.head()

10362
7505


,tmk,in_flood_zone
0,211003003,1
1,211003006,1
2,211003012,1
3,211003030,0
4,211003031,0


### Soil Infiltration

In [19]:
# # Chris' code

# # Pull in soils data from NRCS
# In_polygons = os.path.join("..", "Projected_data/Soils", 'HIstate_nrcs_join2.shp')
# new_col_name = 'Soil_Ksat_2ndry'
# In_points = osds_path

# # Columns you want from the shapefile (Uid should always  be first one) 
# want_cols = ['Uid', 'brockdepmin', 'flodfreqdcd', 'engstafdcd', 'engstafll', 
#              'engstafml', 'cokey', 'ksat_h', 'ksat_l', 'ksat_r', 'sieveno10_r', ]  # 'totalsub_r', slopegradw

# renames = {'brockdepmi':'nrcs_DtoBrock', 
#  'flodfreqdc':'nrcs_FloodFreq',
#  'engstafdcd':'nrcs_septic_dominant', 
#  'engstafll':'nrcs_septic_least', 
#  'engstafml':'nrcs_septic_most', 
#  'ksat_h':'nrcs_ksat_hi',  
#  'ksat_l':'nrcs_ksat_lo',
#  'ksat_r':'nrcs_ksat_rep', 
#  'sieveno10_':'nrcs_rockFrag3in'}    # 'slopegradw':'nrcs_slope', 'totalsub_r':'nrcs_subsidence'

# # Do the spatial join
# arcpy.SpatialJoin_analysis(In_points, In_polygons, os.path.join(tempspace, "test_join_pt_2_poly.shp"))

# # read data from shapefile 
# extracted_points_path = os.path.join(tempspace, "test_join_pt_2_poly.shp")
# arcpy.TableToTable_conversion(extracted_points_path, tempspace, 'extracted_all.csv') # Create a rational file format
# # Create a rational PanDataframe
# Extracted_All = pd.read_csv(os.path.join(tempspace, 'extracted_all.csv'))

# want_cols = ['Uid', 'brockdepmi', 'flodfreqdc', 'engstafdcd', 'engstafll', 
#              'engstafml', 'ksat_h', 'ksat_l', 'ksat_r', 'sieveno10_'] # 'slopegradw' , 'totalsub_r'
# Extract_frame = Extracted_All[want_cols]

# Extract_frame = Extract_frame.replace('NoData', np.nan)  # clean up from arc's trashiness

# numcols = ['brockdepmi', 'ksat_h', 'ksat_l', 'ksat_r', 'sieveno10_' ]  #'slopegradw',  'totalsub_r'
# Extract_frame[numcols] = Extract_frame[numcols].apply(pd.to_numeric, errors='coerce', axis=1)

# Extract_frame.rename(columns=renames, inplace=True)     # Rename to col that you want

# SOILS_NRCS = Extract_frame.copy()


# #### Filling in missing NRCS flood frequency data with rainfall and ksat
# SOILS_NRCS = SOILS_NRCS.merge(RAINFALL, on="Uid", how='left')

# # After the merge, make sure nrcs_FloodFreq is actually NaN (real missing) and not strings like "NoData" or empty strings.
# SOILS_NRCS['nrcs_FloodFreq'] = SOILS_NRCS['nrcs_FloodFreq'].replace('', np.nan)  

# # Set frequents 
# SOILS_NRCS.loc[(SOILS_NRCS['nrcs_FloodFreq'].isnull()) & (SOILS_NRCS['nrcs_ksat_rep'] < 1.1) & (SOILS_NRCS['rainfall_in'] > 135), 'nrcs_FloodFreq'] = "Frequent"
# SOILS_NRCS.loc[(SOILS_NRCS['nrcs_FloodFreq'].isnull()) & (SOILS_NRCS['nrcs_ksat_rep'] < 1.1) & (SOILS_NRCS['rainfall_in'] < 15), 'nrcs_FloodFreq'] = "Frequent"

# # Set Occasionals 
# SOILS_NRCS.loc[(SOILS_NRCS['nrcs_FloodFreq'].isnull()) & (SOILS_NRCS['rainfall_in'] > 135), 'nrcs_FloodFreq'] = "Occasional"
# SOILS_NRCS.loc[(SOILS_NRCS['nrcs_FloodFreq'].isnull()) & (SOILS_NRCS['nrcs_ksat_rep'] < 1.1), 'nrcs_FloodFreq'] = "Occasional"

# # set rest to none
# SOILS_NRCS.loc[SOILS_NRCS['nrcs_FloodFreq'].isnull(), 'nrcs_FloodFreq'] = "None"

# del SOILS_NRCS['rainfall_in']

# SOILS_NRCS.to_csv(os.path.join(intermidiate_DataFramesPath, "SOILS_NRCS.csv"))

In [20]:
soils_gdf = (
    # Load layer
    gpd.read_file(inputs["soils"], layer="soils")
    # Standardize column names to lowercase
    .rename(columns=lambda col: col.lower())
    # Subset to relevant columns from Chris' code
    .loc[:, [
        "ksat_h", "ksat_l",  "ksat_r", "flodfreqdc", "engstafdcd", "engstafll", 
        "engstafml", "sieveno10_", "brockdepmi", "geometry"
    ]]
    # Fix invalid geometries before spatial operations
    .assign(geometry=lambda d: d.geometry.make_valid())
    # Coerce NoData strings to NA + numeric
    .pipe(
        lambda d: d.assign(
            ksat_h=pd.to_numeric(d["ksat_h"], errors="coerce"),
            ksat_l=pd.to_numeric(d["ksat_l"], errors="coerce"),
            ksat_r=pd.to_numeric(d["ksat_r"], errors="coerce"),
            flodfreqdc=pd.to_numeric(d["flodfreqdc"], errors="coerce"),
            sieveno10_=pd.to_numeric(d["sieveno10_"], errors="coerce"),
            brockdepmi=pd.to_numeric(d["brockdepmi"], errors="coerce"),
        )
    )
)
print(len(soils_gdf))
soils_gdf.head()

20743


,ksat_h,ksat_l,ksat_r,flodfreqdc,engstafdcd,engstafll,engstafml,sieveno10_,brockdepmi,geometry
0,16.460000,3.528333,9.866667,NaN,Very limited,Very limited,Very limited,78.333333,84.0,"MULTIPOLYGON (((785636.261 2281696.671, 785612..."
1,15.880000,1.553333,8.666667,NaN,Very limited,Very limited,Very limited,97.500000,NaN,"MULTIPOLYGON (((790415.069 2312700.082, 790423..."
2,NaN,NaN,NaN,NaN,Not rated,Not rated,Not rated,NaN,NaN,"MULTIPOLYGON (((743156.793 2312455.377, 743165..."
3,77.570000,9.170000,39.500000,NaN,Very limited,Very limited,Very limited,78.750000,NaN,"MULTIPOLYGON (((760471.351 2303488.104, 760464..."
4,4.946667,1.346667,3.146667,NaN,Very limited,Not rated,Very limited,31.666667,0.0,"MULTIPOLYGON (((784563.187 2303361.916, 784567..."


In [21]:
# Get ksat values at analysis points (analysis points within soil polygons)
ksat_vals_df = gpd.sjoin(
    analysis_points_gdf.loc[:, ["tmk", "geometry"]],
    # Subset to relevant cols Bob and Johann specified
    soils_gdf.loc[:, ["ksat_h", "ksat_l", "ksat_r", "geometry"]],
    how="left",
    predicate="within",
).drop(columns=["index_right", "geometry"])
print(len(ksat_vals_df))
ksat_vals_df.head()

7505


,tmk,ksat_h,ksat_l,ksat_r
0,211003003,20.076667,6.546667,13.333333
1,211003006,20.076667,6.546667,13.333333
2,211003012,20.076667,6.546667,13.333333
3,211003030,42.000000,14.000000,28.000000
4,211003031,42.000000,14.000000,28.000000


### Distance to Coastline

In [22]:
coastline_gdf = (
    # Load layer
    gpd.read_file(inputs["coastline"], layer="coastline")
    # Subset to geometry column only
    .loc[:, ["geometry"]]
    # Fix invalid geometries before spatial operations
    .assign(geometry=lambda d: d.geometry.make_valid())
    .assign(geometry=lambda d: d.geometry.boundary)
)

# Create a single geometry representing the union of all coastline segments for distance calculations
coastline_union = coastline_gdf.geometry.union_all() 

dist_to_coast_df = (
    analysis_points_gdf.loc[:, ["tmk", "geometry"]]
    .assign(
        # Calculate distance from each analysis point to coastline
        dist_to_coast_m=lambda d: d.geometry.distance(coastline_union),
        dist_to_coast_ft=lambda d: d["dist_to_coast_m"] / FT_TO_M
    )
    .drop(columns=["geometry"])
)
print(len(dist_to_coast_df))
dist_to_coast_df.head()

7505


,tmk,dist_to_coast_m,dist_to_coast_ft
0,211003003,62.315255,204.446376
1,211003006,58.680636,192.521771
2,211003012,83.591376,274.249920
3,211003030,338.897957,1111.869936
4,211003031,264.627792,868.201418


In [23]:
# Threshold for coastal distance calculations
COAST_100FT_M = 100 * FT_TO_M

dists_to_coast_df = (
    analysis_points_gdf.loc[:, ["tmk", "geometry"]]
    .assign(
        # Calculate distance from each analysis point to coastline
        dist_to_coast_m=lambda d: d.geometry.distance(coastline_union),
        dist_to_coast_ft=lambda d: d["dist_to_coast_m"] / FT_TO_M,
        # # Flag analysis points within 100 ft of coastline
        # coast_within_100_ft=lambda d: (d["dist_to_coast_m"] <= COAST_100FT_M).astype("int64"),
    )
    .drop(columns=["geometry"])
)

### Distance to Streams

In [24]:
streams_gdf = (
    # Load layer
    gpd.read_file(inputs["streams"], layer="streams")
    # Subset to geometry column only
    .loc[:, ["geometry"]]
    # Fix invalid geometries before spatial operations
    .assign(geometry=lambda d: d.geometry.make_valid())
)

# Create a single geometry representing the union of all streams segments for distance calculations
streams_union = streams_gdf.geometry.union_all() 

dist_to_streams_df = (
    analysis_points_gdf.loc[:, ["tmk", "geometry"]]
    .assign(
        # Calculate distance from each analysis point to streams
        dist_to_streams_m=lambda d: d.geometry.distance(streams_union),
        dist_to_streams_ft=lambda d: d["dist_to_streams_m"] / FT_TO_M
    )
    .drop(columns=["geometry"])
)
print(len(dist_to_streams_df))
dist_to_streams_df.head()

7505


,tmk,dist_to_streams_m,dist_to_streams_ft
0,211003003,386.051515,1266.573211
1,211003006,223.628149,733.688154
2,211003012,212.389674,696.816516
3,211003030,15.981065,52.431315
4,211003031,96.914820,317.962009


In [25]:
# Threshold for stream distance calculations
STREAM_50FT_M = 50 * FT_TO_M

stream_dist_df = (
    analysis_points_gdf.loc[:, ["tmk", "geometry"]]
    .assign(
        # Calculate distance from each analysis point to streams
        dist_to_streams_m=lambda d: d.geometry.distance(streams_union),
        dist_to_streams_ft=lambda d: d["dist_to_streams_m"] / FT_TO_M,
        # Flag analysis points within 50 ft of streams
        stream_less_than_50_ft=lambda d: (d["dist_to_streams_m"] <= STREAM_50FT_M).astype("int64"),
    )
    .drop(columns=["geometry"])
)

### Distance to Domestic and Municipal Wells

In [26]:
# Domestic wells
wells_dom_gdf = (
    gpd.read_file(inputs["wells_dom"], layer="wells_dom")
    .loc[:, ["geometry"]]
    .assign(geometry=lambda d: d.geometry.make_valid())
)

# Municipal wells
wells_mun_gdf = (
    gpd.read_file(inputs["wells_mun"], layer="wells_mun")
    .loc[:, ["geometry"]]
    .assign(geometry=lambda d: d.geometry.make_valid())
)

# Create single geometries for distance calculations
dom_union = wells_dom_gdf.geometry.union_all()
mun_union = wells_mun_gdf.geometry.union_all()

# Distances from analysis points to nearest well
dist_to_wells_df = (
    analysis_points_gdf.loc[:, ["tmk", "geometry"]]
    .assign(
        dist_to_dom_well_m=lambda d: d.geometry.distance(dom_union),
        dist_to_dom_well_ft=lambda d: d["dist_to_dom_well_m"] / FT_TO_M,
        dist_to_mun_well_m=lambda d: d.geometry.distance(mun_union),
        dist_to_mun_well_ft=lambda d: d["dist_to_mun_well_m"] / FT_TO_M,
    )
    .drop(columns=["geometry"])
)

print(len(dist_to_wells_df))
dist_to_wells_df.head()

c:\Users\apc\AppData\Local\ESRI\conda\envs\matrix_env\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Non-conformant content for record 1 in column WCR, 2006-08-10T00:00:00.0Z, successfully parsed
  return ogr_read(
c:\Users\apc\AppData\Local\ESRI\conda\envs\matrix_env\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Non-conformant content for record 1 in column TEST_DATE, 1998-07-20T00:00:00.0Z, successfully parsed
  return ogr_read(


7505


,tmk,dist_to_dom_well_m,dist_to_dom_well_ft,dist_to_mun_well_m,dist_to_mun_well_ft
0,211003003,7583.276535,24879.516192,1767.215541,5797.951251
1,211003006,7461.652179,24480.486150,1627.535221,5339.682484
2,211003012,7501.156213,24610.092563,1659.757661,5445.399151
3,211003030,7545.432599,24755.356295,1685.444118,5529.672304
4,211003031,7635.997634,25052.485676,1775.304984,5824.491418


### Average Rainfall

In [27]:
rainfall_df = extract_rast_vals(
    in_raster=inputs["rainfall"],
    in_points=analysis_fc,
    col_name="avg_rainfall_in",
    tmk_field="tmk",
    source_units="in",
    output_units="in",
    unit_conversions=UNIT_CONVERSIONS
)
print(len(rainfall_df))
rainfall_df.head()

c:\Users\apc\AppData\Local\ESRI\conda\envs\matrix_env\Lib\site-packages\osgeo\gdal.py:606: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


  Sampled 7,505 points against 'annual_rainfall_hi_hcpt_32604.tif'
  Extracted 7,505 valid values for 'avg_rainfall_in'

7505


,tmk,avg_rainfall_in
0,211003003,106.080597
1,211003006,110.287704
2,211003012,110.287704
3,211003030,121.329597
4,211003031,121.329597


### Land Surface Elevation

In [28]:
dem_df = extract_rast_vals(
    in_raster=inputs["dem"],
    in_points=analysis_fc,
    col_name="land_surface_elev_ft",
    tmk_field="tmk",
    source_units="m",
    output_units="ft",
    unit_conversions=UNIT_CONVERSIONS
)
print(len(dem_df))
dem_df.head()

  Sampled 7,505 points against 'dem_hi_pacioos_mosaic_32604.tif'
  Extracted 7,504 valid values for 'land_surface_elev_ft'
  Converted m -> ft (x3.2808)
  Skipped 1 nodata values

7504


,tmk,land_surface_elev_ft
0,211003003,7.293545
1,211003006,7.958200
2,211003012,13.798596
3,211003030,44.486740
4,211003031,42.756844


### Water Table Elevation

In [29]:
wt_df = extract_rast_vals(
    in_raster=inputs["watertable"],
    in_points=analysis_fc,
    col_name="wt_elev_ft",
    tmk_field="tmk",
    source_units="m",
    output_units="ft",
    unit_conversions=UNIT_CONVERSIONS
)
print(len(wt_df))
wt_df.head()

  Sampled 7,505 points against 'watertable_hi_hcpt_mosaic_32604.tif'
  Extracted 7,023 valid values for 'wt_elev_ft'
  Converted m -> ft (x3.2808)
  Skipped 482 nodata values

7023


,tmk,wt_elev_ft
0,221004084,130.799541
1,221004092,130.048954
2,221004093,148.032311
3,221004098,139.751601
4,221004100,149.123091


### Slope Percentage

Last runtime: 12m 15.1s

In [30]:
# Create slope raster (percent rise)
slope_raster = calculate_slope_percentages(
    in_dem_raster=inputs["dem"],
    out_slope_raster=tempspace / f"{TODAY}_slope_pct.tif",
)

# Sample slope at analysis points
slope_df = extract_rast_vals(
    in_raster=slope_raster,
    in_points=analysis_fc,
    col_name="slope_pct",
    source_units="pct",
    output_units="pct",   # no conversion
)

print(len(slope_df))
slope_df.head()

  Sampled 7,505 points against '20260225_slope_pct.tif'
  Extracted 7,504 valid values for 'slope_pct'
  Skipped 1 nodata values

7504


,tmk,slope_pct
0,211003003,2.190298
1,211003006,2.336785
2,211003012,3.301470
3,211003030,18.649988
4,211003031,22.219120


## MPAT Assembly

### Merge All Relevant Columns

In [31]:
mpat_gdf = (
    # Get parcels with cesspols tmk values and areas
    parcels_gdf[["tmk", "parcel_area_sqft", "geometry"]]
    # Merge cesspool attributes
    .merge(cesspools_df, on="tmk", how="left", validate="one_to_one")
    # Merge building footprint attributes
    .merge(building_fp_parcel_attrs_df, on="tmk", how="left", validate="one_to_one")
    # Merge analysis point source
    .merge(analysis_points_gdf[["tmk", "analysis_point_source"]], on="tmk", how="left", validate="one_to_one")
    # Merge distances from analysis point to nearest SMA
    .merge(dist_to_sma_df[["tmk", "dist_to_sma_ft"]], on="tmk", how="left", validate="one_to_one")
    # Merge soil ksat values at analysis points
    .merge(ksat_vals_df, on="tmk", how="left", validate="one_to_one")
    # Merge distances from analysis points to coast
    .merge(dist_to_coast_df[["tmk", "dist_to_coast_ft"]], on="tmk", how="left", validate="one_to_one")
    # Merge distances from analysis points to nearest stream
    .merge(dist_to_streams_df[["tmk", "dist_to_streams_ft"]], on="tmk", how="left", validate="one_to_one")
    # Merge distances from analysis points to nearest domestic and municipal wells
    .merge(dist_to_wells_df[["tmk", "dist_to_dom_well_ft", "dist_to_mun_well_ft"]], on="tmk", how="left", validate="one_to_one")
    # Merge average rainfall values at analysis points
    .merge(rainfall_df, on="tmk", how="left", validate="one_to_one")
    # Merge land surface elevations at analysis points
    .merge(dem_df, on="tmk", how="left", validate="one_to_one")
    # Merge water table elevations at analysis points
    .merge(wt_df, on="tmk", how="left", validate="one_to_one")
    # Merge slope percentages at analysis points
    .merge(slope_df, on="tmk", how="left", validate="one_to_one")
    # Convert data type from float to integer
    .assign(building_fp_qty=lambda d: d["building_fp_qty"].astype("Int64"))
    # Relocate geometry column to the end
    .pipe(lambda d: d[[c for c in d.columns if c != "geometry"] + ["geometry"]])
)
print(len(mpat_gdf))
mpat_gdf.head()

7505


,tmk,parcel_area_sqft,island,osds_qty,bedroom_qty,building_fp_qty,building_fp_total_area_sqft,analysis_point_source,dist_to_sma_ft,ksat_h,...,ksat_r,dist_to_coast_ft,dist_to_streams_ft,dist_to_dom_well_ft,dist_to_mun_well_ft,avg_rainfall_in,land_surface_elev_ft,wt_elev_ft,slope_pct,geometry
0,211003003,70924.721493,Maui,1,3,2,3445.443843,building_fp_largest_centroid,0.0,20.076667,...,13.333333,204.446376,1266.573211,24879.516192,5797.951251,106.080597,7.293545,NaN,2.190298,"POLYGON ((797198.639 2309554.974, 797196.84 23..."
1,211003006,32346.536954,Maui,1,2,2,2271.544397,building_fp_largest_centroid,0.0,20.076667,...,13.333333,192.521771,733.688154,24480.486150,5339.682484,110.287704,7.958200,NaN,2.336785,"POLYGON ((797225.075 2309335.433, 797159.619 2..."
2,211003012,38854.544707,Maui,1,2,2,1777.038765,building_fp_largest_centroid,0.0,20.076667,...,13.333333,274.249920,696.816516,24610.092563,5445.399151,110.287704,13.798596,NaN,3.301470,"POLYGON ((797153.617 2309339.286, 797107.905 2..."
3,211003030,12086.822566,Maui,1,3,1,661.600642,building_fp_largest_centroid,0.0,42.000000,...,28.000000,1111.869936,52.431315,24755.356295,5529.672304,121.329597,44.486740,NaN,18.649988,"POLYGON ((796828.023 2309063.058, 796822.193 2..."
4,211003031,129395.545175,Maui,1,1,<NA>,NaN,parcel_centroid,0.0,42.000000,...,28.000000,868.201418,317.962009,25052.485676,5824.491418,121.329597,42.756844,NaN,22.219120,"POLYGON ((796820.317 2309089.443, 796830.047 2..."


### Add Classes and Flags

In [32]:
mpat_gdf = (
    mpat_gdf
    # Dervied variables
    .assign(
        # Depth to water table (ft) = land surface elevation - water table elevation
        # Replaces negative values with 0.999
        depth_to_wt_ft=lambda d: (d["land_surface_elev_ft"] - d["wt_elev_ft"]).clip(lower=0.999),
        # Calculate net parcel area (parcel area minus building footprint area)
        net_parcel_area_sqft=lambda d: d.parcel_area_sqft - d.building_fp_total_area_sqft,
    )
    # Classifications
    .assign(
        # Specify lot size requirement classes
        lot_size_req=lambda d: pd.Categorical(
            np.select(
                [
                    d["parcel_area_sqft"] < 10_000,
                    d["parcel_area_sqft"].between(10_000, 21_000, inclusive="both"),
                    d["parcel_area_sqft"] > 21_000,
                ],
                [
                    "Less than 10,000 sqft",
                    "Between 10,000 and 21,000 sqft",
                    "Greater than 21,000 sqft",
                ],
                default=pd.NA,
            ),
            categories=[
                "Less than 10,000 sqft",
                "Between 10,000 and 21,000 sqft",
                "Greater than 21,000 sqft",
            ],
            ordered=True,
        ),
        # Specify depth to watertable suitability
        depth_to_wt_suitability=lambda d: pd.Categorical(
            np.select(
                [
                    d["depth_to_wt_ft"] < 3,
                    d["depth_to_wt_ft"].between(3, 6, inclusive="both"),
                    d["depth_to_wt_ft"] > 6,
                ],
                [
                    "Less than 3 ft",
                    "Between 3 and 6 ft",
                    "Greater than 6 ft",
                ],
                default=pd.NA,
            ),
            categories=[
                "Less than 3 ft",
                "Between 3 and 6 ft",
                "Greater than 6 ft",
            ],
            ordered=True,
        ),
        # Specify SMA constraints (ordered categorical)
        sma_constraints=lambda d: pd.Categorical(
            np.select(
                [
                    d["dist_to_sma_ft"] <= 50,
                    d["dist_to_sma_ft"] > 50,
                ],
                ["Within 50 ft", "Beyond 50 ft"],
                default=pd.NA,
            ),
            categories=["Within 50 ft", "Beyond 50 ft"],
            ordered=True,
        ),
        # Specify climate suitability from avg rainfall (in)
        climate_suitability=lambda d: pd.Categorical(
            np.select(
                [
                    d["avg_rainfall_in"] <= 20,
                    (d["avg_rainfall_in"] > 20) & (d["avg_rainfall_in"] < 75),
                    d["avg_rainfall_in"] >= 75,
                ],
                ["Xeric", "Mesic", "Hydric"],
                default=pd.NA,
            ),
            categories=["Xeric", "Mesic", "Hydric"],
            ordered=True,
        ),
        # Specify slope requirements
        slope_req=lambda d: pd.Categorical(
            np.select(
                [
                    d["slope_pct"] < 8,
                    d["slope_pct"].between(8, 12, inclusive="both"),
                    d["slope_pct"] > 12,
                ],
                [
                    "Slope less than 8%",
                    "Slope between 8 to 12%",
                    "Slope greater than 12%",
                ],
                default=pd.NA,
            ),
            categories=[
                "Slope less than 8%",
                "Slope between 8 to 12%",
                "Slope greater than 12%",
            ],
            ordered=True,
        ),
    )
    # Flags
    # Analysis point in flood zone
    .merge(flood_flag_df, on="tmk", how="left", validate="one_to_one")
    .assign(
        # Analysis point in SMA (distance == 0 ft)
        in_sma=lambda d: (d["dist_to_sma_ft"] == 0.0).astype("int64"),
        # Analysis point within 100 ft of coastline
        coast_within_100_ft=lambda d: (d["dist_to_coast_ft"] <= 100 * FT_TO_M).astype("int64"),
    )
    # Relocate geometry column to the end
    .pipe(lambda d: d[[c for c in d.columns if c != "geometry"] + ["geometry"]])
)
print(len(mpat_gdf))
mpat_gdf.head()

7505


,tmk,parcel_area_sqft,island,osds_qty,bedroom_qty,building_fp_qty,building_fp_total_area_sqft,analysis_point_source,dist_to_sma_ft,ksat_h,...,net_parcel_area_sqft,lot_size_req,depth_to_wt_suitability,sma_constraints,climate_suitability,slope_req,in_flood_zone,in_sma,coast_within_100_ft,geometry
0,211003003,70924.721493,Maui,1,3,2,3445.443843,building_fp_largest_centroid,0.0,20.076667,...,67479.277650,"Greater than 21,000 sqft",NaN,Within 50 ft,Hydric,Slope less than 8%,1,1,0,"POLYGON ((797198.639 2309554.974, 797196.84 23..."
1,211003006,32346.536954,Maui,1,2,2,2271.544397,building_fp_largest_centroid,0.0,20.076667,...,30074.992557,"Greater than 21,000 sqft",NaN,Within 50 ft,Hydric,Slope less than 8%,1,1,0,"POLYGON ((797225.075 2309335.433, 797159.619 2..."
2,211003012,38854.544707,Maui,1,2,2,1777.038765,building_fp_largest_centroid,0.0,20.076667,...,37077.505942,"Greater than 21,000 sqft",NaN,Within 50 ft,Hydric,Slope less than 8%,1,1,0,"POLYGON ((797153.617 2309339.286, 797107.905 2..."
3,211003030,12086.822566,Maui,1,3,1,661.600642,building_fp_largest_centroid,0.0,42.000000,...,11425.221924,"Between 10,000 and 21,000 sqft",NaN,Within 50 ft,Hydric,Slope greater than 12%,0,1,0,"POLYGON ((796828.023 2309063.058, 796822.193 2..."
4,211003031,129395.545175,Maui,1,1,<NA>,NaN,parcel_centroid,0.0,42.000000,...,NaN,"Greater than 21,000 sqft",NaN,Within 50 ft,Hydric,Slope greater than 12%,0,1,0,"POLYGON ((796820.317 2309089.443, 796830.047 2..."


In [33]:
# Re-arrange columns
front = [
    "island", "tmk", 
    "osds_qty", "bedroom_qty", "building_fp_qty", 
    "parcel_area_sqft", "building_fp_total_area_sqft", "net_parcel_area_sqft",
]

dists = [
    "dist_to_sma_ft", "dist_to_coast_ft", "dist_to_streams_ft",
    "dist_to_dom_well_ft", "dist_to_mun_well_ft",
]

env = [
    "ksat_h", "ksat_l", "ksat_r",
    "avg_rainfall_in", "land_surface_elev_ft", "wt_elev_ft", "depth_to_wt_ft",
    "slope_pct",
]

classes = [
    "lot_size_req", "depth_to_wt_suitability", "sma_constraints",
    "climate_suitability", "slope_req",
]

flags = ["in_flood_zone", "in_sma", "coast_within_100_ft"]

end_cols = ["analysis_point_source", "geometry"]

desired = front + dists + env + classes + flags + end_cols

mpat_gdf = mpat_gdf[[c for c in desired if c in mpat_gdf.columns]]
mpat_gdf.head()

,island,tmk,osds_qty,bedroom_qty,building_fp_qty,parcel_area_sqft,building_fp_total_area_sqft,net_parcel_area_sqft,dist_to_sma_ft,dist_to_coast_ft,...,lot_size_req,depth_to_wt_suitability,sma_constraints,climate_suitability,slope_req,in_flood_zone,in_sma,coast_within_100_ft,analysis_point_source,geometry
0,Maui,211003003,1,3,2,70924.721493,3445.443843,67479.277650,0.0,204.446376,...,"Greater than 21,000 sqft",NaN,Within 50 ft,Hydric,Slope less than 8%,1,1,0,building_fp_largest_centroid,"POLYGON ((797198.639 2309554.974, 797196.84 23..."
1,Maui,211003006,1,2,2,32346.536954,2271.544397,30074.992557,0.0,192.521771,...,"Greater than 21,000 sqft",NaN,Within 50 ft,Hydric,Slope less than 8%,1,1,0,building_fp_largest_centroid,"POLYGON ((797225.075 2309335.433, 797159.619 2..."
2,Maui,211003012,1,2,2,38854.544707,1777.038765,37077.505942,0.0,274.249920,...,"Greater than 21,000 sqft",NaN,Within 50 ft,Hydric,Slope less than 8%,1,1,0,building_fp_largest_centroid,"POLYGON ((797153.617 2309339.286, 797107.905 2..."
3,Maui,211003030,1,3,1,12086.822566,661.600642,11425.221924,0.0,1111.869936,...,"Between 10,000 and 21,000 sqft",NaN,Within 50 ft,Hydric,Slope greater than 12%,0,1,0,building_fp_largest_centroid,"POLYGON ((796828.023 2309063.058, 796822.193 2..."
4,Maui,211003031,1,1,<NA>,129395.545175,NaN,NaN,0.0,868.201418,...,"Greater than 21,000 sqft",NaN,Within 50 ft,Hydric,Slope greater than 12%,0,1,0,parcel_centroid,"POLYGON ((796820.317 2309089.443, 796830.047 2..."


## Export

In [34]:
# Ensure output dir exists
output_dir.mkdir(parents=True, exist_ok=True)

# Export MPAT as GeoPackage
mpat_layer = f"mpat_v{VERSION}"
mpat_gdf.to_file(outputs["mpat_gpkg"], layer=mpat_layer, driver="GPKG")
print("Wrote GPKG:", outputs["mpat_gpkg"])

# Export MPAT as CSV (drops geometry)
mpat_csv_df = mpat_gdf.drop(columns=["geometry"], errors="ignore")
mpat_csv_df.to_csv(outputs["mpat_csv"], index=False)
print("Wrote CSV:", outputs["mpat_csv"])

Wrote GPKG: f:\projects\shuler_lab_projects\HiOSDS-TechSuitabilityAnalysis\data\03_processed\mpat_v02_32604.gpkg
Wrote CSV: f:\projects\shuler_lab_projects\HiOSDS-TechSuitabilityAnalysis\data\03_processed\mpat_v02.csv
